<a href="https://colab.research.google.com/github/ArturSouza293/Datasets/blob/master/Medidas_de_Complexidade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Medidas de Complexidade LORENA ET AL

##Primeiro Grupo de Funções

##Serão as funções auxilires 

01 - Normalização

        Vai normalizar as colunas do conjuntos de dados (dentro [0,1])

In [0]:
Normalize = function(dataset) {
    dataset = as.matrix(dataset)
    numberColumn = ncol(dataset)

    for (column in 1:numberColumn)
        dataset[,column] = (dataset[,column] - min(dataset[,column])) /
                           (max(dataset[,column]) - min(dataset[,column]))

    dataset
}

02 - Formata o Conjunto de Dados
      
       Formatar o conjunto de dados de entrada 

In [0]:
FormatDataset = function(dataset, output){
    
    dataset = as.matrix(dataset)
    numberColumn = ncol(dataset)
  
    if(!is.null(output)){
        input = dataset
    } else {
        input  = as.matrix(dataset[,-numberColumn])
        output = as.matrix(dataset[,numberColumn])
        numberColumn = ncol(input)
    }

    list(input = Normalize(input), output = Normalize(output), 
        numberColumn = numberColumn, numberRows = nrow(input))
}

03 - Posição Max
  
        Indice do elemento maior

In [0]:
MaxPosition = function(array) order(-array)[1]

04 - Posição Min
  
        inidice do elemento menor

In [0]:
MinPosition = function(array) order(array)[1]

05 - Correlacao de Spearman

        Computa a Correlação de Spearman entre a diferenca de dois ranks (que não tenham "conexao")

In [0]:
spearman_from_rank = function(rank){
  size=length(rank)
  results=1-6*sum(rank^2)/(size^3-size)
  results
}

06 - Remocao de exemplos de numeros (??)
  
        Remove exemplos do conjuto ate que uma correlacao especifica ao output seja atingida 

In [0]:
ExamplesRemovedNumber = function(x,y,minCorrelation)
{
  
  numberRows = length(x)
  if(numberRows == length(y))
  {
    remainingRows = numberRows
    maxPosition = 0
    xorder=rank(x)
    yorder=rank(y)
    diff=xorder-yorder
    
    correlation = spearman_from_rank(diff)
    
    if(correlation < 0){
      yorder=rank(-y)
      diff=xorder-yorder
      correlation = spearman_from_rank(diff)
    }
    
    while(abs(correlation) < minCorrelation && !is.na(correlation))
    {
      
      maxPosition = which.max(abs(diff))
      
      diff=diff +
        ((yorder>yorder[maxPosition]) -
           (xorder>xorder[maxPosition]))
      
      yorder=yorder[-maxPosition]
      xorder=xorder[-maxPosition] 
      diff=diff[-maxPosition]
      remainingRows = remainingRows - 1
      correlation = spearman_from_rank(diff)
      
      if(is.na(correlation))
        correlation
    }
    
    (numberRows-remainingRows)/numberRows
  } else NA
}

## Segundo Grupo de Funcoes 

## Medidas de Complexidade 

### A explicação de cada medida está no Artigo Completo 

07 - C1 - Maximum Feature Correlation

                




In [0]:
C1 = function(dataset, output = NULL) {   
  
  formatedDataset = FormatDataset(dataset, output)
  input = formatedDataset$input
  output = formatedDataset$output
  numberColumn = formatedDataset$numberColumn
  
  correlations = array(0,numberColumn)
  
  for (column in 1:numberColumn){
    correlation = cor(output, input[,column], method = "spearman")
    
    if(!is.na(correlation))
      correlations[column] = abs(correlation)
  }
  
  max(correlations)
}

08 - C2 - Average Feature Correlation to the Output
  

In [0]:
C2 = function(dataset, output = NULL)
{   
    formatedDataset = FormatDataset(dataset, output)
    input = formatedDataset$input
    output = formatedDataset$output
    numberColumn = formatedDataset$numberColumn

    correlations = array(0,numberColumn)
    
    for (column in 1:numberColumn)
        correlations[column] = abs(cor(output, input[,column], 
            method = "spearman"))

    naRemove = !is.na(correlations)

    mean(correlations[naRemove])
}

09 - C3 - Individual Feature Efficiency

In [0]:
C3 = function(dataset, output = NULL, minCorrelation = 0.9)
{
    formatedDataset = FormatDataset(dataset, output)
    input = formatedDataset$input
    output = formatedDataset$output
    numberColumn = formatedDataset$numberColumn

    correlations = array(1,numberColumn)
    
    for (column in 1:numberColumn)
        correlations[column] = ExamplesRemovedNumber(output, input[,column], minCorrelation)
    
    naRemove = !is.na(correlations)
    
    min(correlations[naRemove])
}

10 - C4 - Collective Feature Efficiency

In [0]:
C4 = function(dataset, output = NULL, minResidual = 0.1){
    
    formatedDataset = FormatDataset(dataset, output)
    input = formatedDataset$input
    output = formatedDataset$output
    numberColumn = formatedDataset$numberColumn
    numberRows = formatedDataset$numberRows

    stop = FALSE
    looked = 0
    
    while(!stop){
        
        correlations = cor(output,input,
                             method="spearman")
        indexMostCorrelated = MaxPosition(abs(correlations))
        if(!is.na(indexMostCorrelated)){
            
            looked = looked + 1
            linearModel = lm(output~input[,indexMostCorrelated])
            indexRemove = abs(linearModel$residuals) > minResidual
            input = input[indexRemove,]
            output = output[indexRemove]
            if(sum(!indexRemove) == length(indexRemove) | length(output)==1 |  looked == numberColumn)
                stop = TRUE
        }
    }
    if(length(output) == 1)
       0
    else
       length(output)/numberRows
}

11 - L1 - Distance of the Data Itens to the Linear Function

In [0]:
L1 = function(dataset, output = NULL){
    
    formatedDataset = FormatDataset(dataset, output)
    input = formatedDataset$input
    output = formatedDataset$output
    naRemove = !is.na(cor(output, input, method = "spearman"))
    numberColumns = formatedDataset$numberColumn
    if(numberColumns > 1)
        linearModel = lm(output~input[,naRemove])
    else 
        linearModel = lm(output~input[])
    mean(abs(linearModel$residuals))
}

12 - L2 - Average Error of Linear Regressor

In [0]:
L2 = function(dataset, output = NULL){
    
    formatedDataset = FormatDataset(dataset, output)
    input = formatedDataset$input
    output = formatedDataset$output

    naRemove = !is.na(cor(output, input, method = "spearman"))

    numberColumns = formatedDataset$numberColumn
    if(numberColumns > 1)
      linearModel = lm(output~input[,naRemove])
    else 
      linearModel = lm(output~input[])
        
    mean(linearModel$residuals^2)
}

13 - S1 - Output Distribution 

In [0]:
S1 = function(dataset, output = NULL){
    
    formatedDataset = FormatDataset(dataset, output)
    input = formatedDataset$input
    output = formatedDataset$output
    numberRows = formatedDataset$numberRows
    
    fullGraph = graph.full(numberRows, directed = FALSE, loops = FALSE)
    E(fullGraph)$weight= dist(input, method = "euclidian")
    mst = minimum.spanning.tree(fullGraph, algorithm = "prim")
    edgelist = get.edgelist(mst);
    mean(abs(output[edgelist[,1]] - output[edgelist[,2]]))
}


14 - S2 - Input Distribution

In [0]:
S2  = function(dataset, output = NULL){
    
    getDistances = function(dataset, numberRows, numberColumns){

        distances = array(0,numberRows-1)
        for(line in 2:numberRows){
            if(numberColumns > 1)
               distances[line-1] = dist(dataset[(line-1):line,])
            else
               distances[line-1] = dist(dataset[(line-1):line])
          } 
        distances
    }

    formatedDataset = FormatDataset(dataset, output)
    input = formatedDataset$input
    output = formatedDataset$output
    numberRows = formatedDataset$numberRows
    numberColumns = formatedDataset$numberColumn
   
    order = order(output)
    if(numberColumns > 1)
        distances = getDistances(input[order,], numberRows,numberColumns)
    else
        distances = getDistances(input[order], numberRows,numberColumns)
    
    mean(distances)
}

15 - S3 - Error of a Nearest Neighbor regressor 

In [0]:
S3  = function(dataset, output = NULL){

    formatedDataset = FormatDataset(dataset, output)
    input = formatedDataset$input
    output = formatedDataset$output
    numberRows = formatedDataset$numberRows
    predictions = matrix(0,numberRows)
    distances = as.matrix(dist(input, method = "euclidian",diag=TRUE,upper=TRUE))

    diag(distances) <- Inf
    for(line in 1:numberRows){
        predictions[line] = output[MinPosition(distances[line,,drop=FALSE])]
    }
    mean((predictions-output)^2)
}

16 - L3 - Non-linearity of a Linear regressor

In [0]:
L3 = function(dataset, output = NULL){

    formatedDataset = FormatDataset(dataset, output)
    input = formatedDataset$input
    output = formatedDataset$output
    numberRows = formatedDataset$numberRows
    naRemove = !is.na(cor(output, input, method = "spearman"))
    numberColumn = sum(naRemove)

    order = order(output)
      
    output = output[order] 
    numberColumns = formatedDataset$numberColumn
    randomUniform = runif(numberRows - 1)
    
    if(numberColumns > 1){
      input = input[order,]  
      linearModel = lm(output~input[,naRemove])
      newInput = randomUniform*input[2:numberRows-1,naRemove] + 
        (1-randomUniform)*input[2:numberRows,naRemove]
      
    }
    else{
      input = input[order]
      linearModel = lm(output~input[])
      newInput = randomUniform*input[2:numberRows-1] + 
        (1-randomUniform)*input[2:numberRows]
      
    }
    newOutput = randomUniform*output[2:numberRows-1] + 
               (1-randomUniform)*output[2:numberRows]
    newPredict = predict.lm(linearModel,newdata = as.data.frame(input<-newInput))
    
    mean((newPredict - newOutput)^2)
}

17 - S4 - Non-linearity of Nearest Neighbor regressor

In [0]:
S4 = function(dataset, output = NULL){

    formatedDataset = FormatDataset(dataset, output)
    input = formatedDataset$input
    output = formatedDataset$output
    numberRows = formatedDataset$numberRows
    naRemove = !is.na(cor(output, input, method = "spearman"))
    numberColumn = sum(naRemove)

    order = order(output)
    output = output[order,] 
    numberColumns = formatedDataset$numberColumn
    randomUniform = runif(numberRows - 1)
    
    if(numberColumns > 1){
      input = input[order,]  
      newInput = randomUniform*input[2:numberRows-1,] + 
        (1-randomUniform)*input[2:numberRows,]
      
          }
    else{
      input = input[order]
      newInput = randomUniform*input[2:numberRows-1] + 
        (1-randomUniform)*input[2:numberRows]
      
      }
    newOutput = randomUniform*output[2:numberRows-1] + 
               (1-randomUniform)*output[2:numberRows]
    newPredict = knn.reg(as.data.frame(input), as.data.frame(newInput), output, k = 1)$pred
    mean((newPredict - newOutput)^2)
}

18 - T2 Average Number of Examples per dimension

In [0]:
T2 = function(dataset, output = NULL){
    formatedDataset = FormatDataset(dataset, output)
    (formatedDataset$numberRows / formatedDataset$numberColumn)
}

LINK DO GIT : https://github.com/aclorena/ComplexityRegression/blob/master/measures.r